In [ ]:
from google.colab import files
import pandas as pd

# Upload CSV file
uploaded = files.upload()

# Automatically get the uploaded filename
filename = list(uploaded.keys())[0]

# Load CSV into a DataFrame
df = pd.read_csv(filename)

# Show first 5 rows to check
df.head()

Saving noncommunicable_diseases_btn_raw (3).csv to noncommunicable_diseases_btn_raw (3).csv


,GHO (CODE),GHO (DISPLAY),GHO (URL),YEAR (DISPLAY),STARTYEAR,ENDYEAR,REGION (CODE),REGION (DISPLAY),COUNTRY (CODE),COUNTRY (DISPLAY),DIMENSION (TYPE),DIMENSION (CODE),DIMENSION (NAME),Numeric,Value,Low,High
0,#indicator+code,#indicator+name,#indicator+url,#date+year,#date+year+start,#date+year+end,#region+code,#region+name,#country+code,#country+name,#dimension+type,#dimension+code,#dimension+name,#indicator+value+num,#indicator+value,#indicator+value+low,#indicator+value+high
1,NCD_CHOL_MEANNONHDL_A,"Mean Non-HDL cholesterol, age-standardized",https://www.who.int/data/gho/data/indicators/i...,1991,1991,1991,SEAR,South-East Asia,BTN,Bhutan,SEX,SEX_FMLE,Female,3,3.0 [2.7-3.3],2.7,3.3
2,SA_0000001400,"Alcohol, recorded per capita (15+) consumption...",https://www.who.int/data/gho/data/indicators/i...,1989,1989,1989,SEAR,South-East Asia,BTN,Bhutan,ALCOHOLTYPE,ALCOHOLTYPE_SA_BEER,Beer,0.31,0.31,NaN,NaN
3,SA_0000001747,"Alcohol, recorded per capita (15+) consumption...",https://www.who.int/data/gho/data/indicators/i...,2020,2020,2020,SEAR,South-East Asia,BTN,Bhutan,NaN,NaN,NaN,2.719382025,2.7 [1.9 - 3.5],1.90229731,3.542702365
4,NCD_BMI_PLUS1C,Prevalence of overweight among children and ad...,https://www.who.int/data/gho/data/indicators/i...,2010,2010,2010,SEAR,South-East Asia,BTN,Bhutan,SEX,SEX_MLE,Male,9.0024188,9.0 [3.5-18.0],3.46319,18.038097


In [ ]:
import numpy as np

# 1️⃣ Drop first row (metadata)
df = df.drop(0).reset_index(drop=True)

# 2️⃣ Clean column names
df.columns = (
    df.columns
      .str.strip()
      .str.replace(r"[#,+()]", "", regex=True)
      .str.replace(" ", "_")
)

# 3️⃣ Convert year columns to numeric
for col in ["YEAR_DISPLAY", "STARTYEAR", "ENDYEAR"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

# 4️⃣ Extract numeric part from 'Value' column
def extract_numeric(value):
    if isinstance(value, str):
        first_part = value.split()[0]
        try:
            return float(first_part)
        except:
            return np.nan
    return value

if "Value" in df.columns:
    df["Value_num"] = df["Value"].apply(extract_numeric)

# 5️⃣ Convert numeric columns to floats
for col in ["Numeric", "Low", "High", "Value_num"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 6️⃣ Drop fully empty columns
df = df.dropna(axis=1, how="all")

# 7️⃣ Replace blank strings with NaN
df = df.replace("", np.nan)

# 8️⃣ Keep only rows with actual Value_num
df = df[df["Value_num"].notna()].reset_index(drop=True)

# 9️⃣ Keep only important columns
columns_to_keep = ["COUNTRY_DISPLAY", "YEAR_DISPLAY", "STARTYEAR", "ENDYEAR", "GHO_DISPLAY", "DIMENSION_NAME", "Value_num"]
df_cleaned = df[columns_to_keep].copy()

# 1️⃣0️⃣ Check cleaned data
df_cleaned.head(), df_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3714 entries, 0 to 3713
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   COUNTRY_DISPLAY  3714 non-null   object 
 1   YEAR_DISPLAY     3714 non-null   Int64  
 2   STARTYEAR        3714 non-null   Int64  
 3   ENDYEAR          3714 non-null   Int64  
 4   GHO_DISPLAY      3714 non-null   object 
 5   DIMENSION_NAME   3659 non-null   object 
 6   Value_num        3714 non-null   float64
dtypes: Int64(3), float64(1), object(3)
memory usage: 214.1+ KB


(  COUNTRY_DISPLAY  YEAR_DISPLAY  STARTYEAR  ENDYEAR  \
 0          Bhutan          1991       1991     1991   
 1          Bhutan          1989       1989     1989   
 2          Bhutan          2020       2020     2020   
 3          Bhutan          2010       2010     2010   
 4          Bhutan          2012       2012     2012   
 
                                          GHO_DISPLAY DIMENSION_NAME  Value_num  
 0         Mean Non-HDL cholesterol, age-standardized         Female       3.00  
 1  Alcohol, recorded per capita (15+) consumption...           Beer       0.31  
 2  Alcohol, recorded per capita (15+) consumption...            NaN       2.70  
 3  Prevalence of overweight among children and ad...           Male       9.00  
 4  Premature deaths due to noncommunicable diseas...           Male      49.00  ,
 None)

In [ ]:
df_cleaned.to_csv("cleaned_data.csv", index=False)
from google.colab import files
files.download("cleaned_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>